In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.autograd.functional import hessian, jacobian

In [6]:
x = np.linspace(0,1,10, dtype=np.float)
t = np.linspace(0,1,10, dtype=np.float)
x, t = np.meshgrid(x, t) 
x = x.flatten()
t = t.flatten()
X = np.concatenate((x.reshape(-1,1), t.reshape(-1, 1)), axis=1)

num_points, input_dim = X.shape
hidden_neurons = 100
output_dim = 1

In [48]:
def g_bc(x, t):
    return (1 - t) * torch.sin(np.pi * x)

def g_nn(x, t, N):
    return x * (1 - x) * t * N

def g_trial(val, N):
    x, t = val.T
    return g_bc(x, t) + g_nn(x, t, N)

def ode_loss(x, N):
    #dudx2 = hessian(g_trial, (x, N))
    #dudt = jacobian(g_trial, (x, N))
    
    return torch.sum(g_trial(x, N))

# https://pytorch.org/docs/stable/autograd.html#torch.autograd.functional.hessian
#u_hess = hessian(g_trial, (x, N))
# https://pytorch.org/docs/stable/autograd.html#torch.autograd.functional.hessian
#u_jacob = jacobian(g_trial, (x, N))

torch.autograd.set_detect_anomaly(True)

In [49]:
dtype = torch.float
#device = torch.device("cuda:0")
device = torch.device("cpu")

x = torch.from_numpy(X)
x = x.to(dtype)

# Create random Tensors for weights.
w1 = torch.randn(input_dim, hidden_neurons, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden_neurons, output_dim, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-7
for t in range(5):
    N_output = F.relu(x.mm(w1)).mm(w2)
    
    # Compute and print loss
    loss = ode_loss(x, N_output)
    print(loss)
    # Use autograd to compute the backward pass.
    loss.backward()
    # Update weights using gradient descent
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()


tensor(2665.4158, grad_fn=<SumBackward0>)
tensor(2663.0588, grad_fn=<SumBackward0>)
tensor(2660.7019, grad_fn=<SumBackward0>)
tensor(2658.3450, grad_fn=<SumBackward0>)
tensor(2655.9885, grad_fn=<SumBackward0>)
